TODO
Correction for SepReformerSettings sr duplicated

In [1]:
import os
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
workdir, _ = os.path.split(os.path.abspath(os.getcwd()))
input = os.path.join(os.getcwd(), 'SepReformer/sample_wav/sample_WSJ.wav')

In [4]:
from simple_vg.vg_types import AudioSetting

settings: AudioSetting = {
    'mono': True,
    'n_fft': None,
    'sr': 16000,
    'use_torch': True,
    'window_size': None,
    'opt_settings': None
}


In [5]:
from simple_vg.process_audio import LoadAudioFile, LoadAudioSettings, LoadAudioBackends

load_settings: LoadAudioSettings = {
    'backend': LoadAudioBackends.TORCH_SOX,
    'effects': [],
    'channel_first': True
}

settings['opt_settings'] = load_settings

load_audio = LoadAudioFile(settings)

In [6]:
from simple_vg.simple_elements import ClosureElement

only_wav = ClosureElement(lambda _, inp: inp[0], None)

In [7]:
from simple_vg.speech_separation.sepreformer import *

sep_model_settings: SepReformerSetting = {
    'batch_size': 1,
    'chunk_max_len': None,
    'distributed_gpu': False,
    'model': SepReformerModels.SepReformer_Base_WSJ0,
    'n_speaker': 2,
    'sr': 16000
}


In [8]:
from simple_vg.process_overlap import *

sep_settings: SeparateOverlappedSettings = {
    'model_settings': sep_model_settings,
    'model': SeparateOverlappedModels.SepReformer
}

overlap_settings: ProcessOverlappedAudioSettings = {
    'tasks': ProcessOverlappedAudioTasks.SEPARATE_AUDIO,
    'separation_settings': sep_settings
}

settings['opt_settings'] = overlap_settings
overlap_elem = ProcessOverlappedAudio(settings)

In [9]:
import sys
from simple_vg.pipeline import VGPipeline

pipeline_elems = [load_audio, only_wav, overlap_elem]
pipeline = VGPipeline()
pipeline.sequential(pipeline_elems)
pipeline.run(input=input, dbg=True)


Index-0 LoadAudioFile Pipeline Output: tuple, shape: None
Index-1 ClosureElement Pipeline Output: Tensor, shape: torch.Size([1, 147186])
Index-2 ProcessOverlappedAudio Pipeline Output: Tensor, shape: torch.Size([1, 2, 1, 147184])


In [10]:
overlap_elem.model.dataset.curr_data_idx = 0
overlap_elem.model.dataset.curr_data_idx = 0


In [11]:
a = overlap_elem.model.dataset.__getitem__(0)

In [12]:
a.shape

torch.Size([147186])